In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Data Processing

In [ ]:
import numpy as np
import pandas as pd

# Define the file paths
file_path1 = '/content/drive/MyDrive/SuperUROP /Data Analysis/caltech_training_data.csv'
file_path2 = '/content/drive/MyDrive/SuperUROP /Data Analysis/caltech_testing_data.csv'
file_path3 = '/content/drive/MyDrive/SuperUROP /Data Analysis/JPL_training_data.csv'
file_path4  = '/content/drive/MyDrive/SuperUROP /Data Analysis/JPL_testing_data.csv'
# Use pandas to read the CSV files and then convert them to NumPy arrays
caltech_train = pd.read_csv(file_path1).values
caltech_test = pd.read_csv(file_path2).values

JPL_train = pd.read_csv(file_path3).values
JPL_test=pd.read_csv(file_path4).values

In [ ]:
#Remove row number (in 1st column)
caltech_train=caltech_train[:,1:]
caltech_test=caltech_test[:,1:]

JPL_train=JPL_train[:,1:]
JPL_test=JPL_test[:,1:]

In [ ]:
#Remove departure time (2nd column)
# Convert arrival date to hour and find day of the week
from datetime import datetime

def convert_time_and_day(data_array):
    """
    Converts the time from HH:MM to HH.XX format and appends the day of the week to it.
    Also, removes the second column.
    """
    transformed_data = []
    for row in data_array:
        # Convert the arrival time to HH.XX format
        time_obj = datetime.strptime(row[0], '%Y-%m-%d %H:%M:%S')
        new_time = time_obj.hour + (time_obj.minute / 60.0)

        # Convert the date to a day of the week
        day_of_week = time_obj.strftime('%A')
        new_time = str(new_time) + " " + day_of_week

        # Create a new row excluding the second column
        new_row = [new_time] + list(row[2:])
        transformed_data.append(new_row)

    return np.array(transformed_data)

caltech_train=convert_time_and_day(caltech_train)
caltech_test=convert_time_and_day(caltech_test)
JPL_train=convert_time_and_day(JPL_train)
JPL_test=convert_time_and_day(JPL_test)

In [ ]:
def day_to_number(day):
    """Converts a day of the week to its corresponding discrete value."""
    days = {
        'Monday': 1,
        'Tuesday': 2,
        'Wednesday': 3,
        'Thursday': 4,
        'Friday': 5,
        'Saturday': 6,
        'Sunday': 7
    }
    return days[day]

def separate_time_and_day(data_array):
    """
    Separates the time and day in the given column,
    and converts the day into a discrete value between 1 and 7.

    """
    transformed_data = []
    for row in data_array:
        time_day_str = row[0]
        time, day = time_day_str.split()
        time = float(time)
        day_num = day_to_number(day)

        # Create a new row with separated time and day number
        new_row = [time, day_num] + list(row[1:])
        transformed_data.append(new_row)

    return np.array(transformed_data)

caltech_train=separate_time_and_day(caltech_train)
caltech_test=separate_time_and_day(caltech_test)
JPL_train=separate_time_and_day(JPL_train)
JPL_test=separate_time_and_day(JPL_test)

#Discretize the stay duration and energy consumption

In [ ]:
caltech_train=caltech_train[:,[2,3,4]]
JPL_train=JPL_train[:,[0,3,4]]

In [ ]:
#Calcuate max duration and energy consumption (used for sparsity matrix dimension)
df_caltech = pd.DataFrame(caltech_train, columns=['Energy Consumption', 'UserID', 'Stay Duration'])
max_energy_caltech = int(np.round(df_caltech['Energy Consumption'].astype(float).max()))
max_duration_caltech = int(np.round(df_caltech['Stay Duration'].astype(float).max()))

df_JPL = pd.DataFrame(JPL_train, columns=['Energy Consumption', 'UserID', 'Stay Duration'])
max_energy_JPL = int(np.round(df_JPL['Energy Consumption'].astype(float).max()))
max_duration_JPL = int(np.round(df_JPL['Stay Duration'].astype(float).max()))

In [ ]:
import numpy as np

# Convert columns 0 and 2 to float
caltech_train[:, 0] = caltech_train[:, 0].astype(float)
caltech_train[:, 2] = caltech_train[:, 2].astype(float)
JPL_train[:, 0] = JPL_train[:, 0].astype(float)
JPL_train[:, 2] = JPL_train[:, 2].astype(float)

# Discretize stay duration into 48 half-hour intervals
caltech_train[:, 2] = np.floor(caltech_train[:, 2].astype(float) * 2).astype(int).astype(str)
JPL_train[:, 2] = np.floor(JPL_train[:, 2].astype(float) * 2).astype(int).astype(str)

#Discretize energy consumption into 1 kWh intervals (rounded to nearest ineteger)
caltech_train[:, 0] = np.round(caltech_train[:, 0].astype(float)).astype(int).astype(str)
JPL_train[:, 0] = np.round(JPL_train[:, 0].astype(float)).astype(int).astype(str)

#Calculate the joint probability distribution of stay duration and energy consumption for each user

In [ ]:
import numpy as np
import pandas as pd

def compute_user_joint_distributions(data_array):
    # Convert array to DataFrame
    df = pd.DataFrame(data_array, columns=['Energy Consumption', 'UserID', 'Stay Duration'])

    # Find the max value in the 'Arrival Time' column after converting to float
    max_arrival_time = int(np.round(df['Energy Consumption'].astype(float).max()))

    # Create a range for expected values based on max_arrival_time
    all_times = [str(i) for i in range(max_arrival_time + 1)]

    # Split the data based on the UserID
    user_datasets = {user_id: user_data for user_id, user_data in df.groupby('UserID')}

    # Inner function to compute the 2D joint distribution for a single user dataset
    def compute_joint_dist(user_data):
        joint_matrix = user_data.pivot_table(index='Stay Duration', columns='Energy Consumption', values='UserID', aggfunc='size').fillna(0)

        # Reindex to ensure all values are present
        joint_matrix = joint_matrix.reindex(index=all_times, columns=[str(i) for i in range(48)], fill_value=0)

        return joint_matrix / joint_matrix.sum().sum()

    # Construct joint probability distribution for each user
    joint_distributions = {user_id: compute_joint_dist(data) for user_id, data in user_datasets.items()}

    return joint_distributions

joint_dists_caltech = compute_user_joint_distributions(caltech_train)
joint_dists_JPL=compute_user_joint_distributions(JPL_train)

#Calculate the joint entropy for stay duration and energy consumption for each user

In [ ]:
import pandas as pd
import numpy as np

def joint_entropy(matrix):
    # Normalize the matrix
    prob_matrix = matrix / matrix.sum().sum()

    # Compute the joint entropy
    prob_matrix = prob_matrix.to_numpy()  # Convert DataFrame to numpy array for efficient computation
    log_prob_matrix = np.log2(prob_matrix, where=(prob_matrix > 0))
    log_prob_matrix[prob_matrix <= 0] = 0  # Set log(0) to 0
    entropy = -np.sum(prob_matrix * log_prob_matrix)

    return entropy

entropies_caltech = {user: joint_entropy(matrix) for user, matrix in joint_dists_caltech.items()}
entropies_JPL = {user: joint_entropy(matrix) for user, matrix in joint_dists_JPL.items()}

In [ ]:
entropies_JPL

#Calculate the sparsity for stay duration and energy consumption dataset for each user
-The matrix is (max_duration+1) columns and (max_energy+1) rows

-Sparsity calculated for each user using the same size of matrix because use the max duration and energy of the entire dataset not each user -> same as done in the paper

In [ ]:
import pandas as pd

def sparsity(matrix, max_duration,max_energy):
    total_elements = (max_energy+1) * (max_duration+1) #matrix is a (energy+1)*(duration+1) element (+1 for 0 index)
    non_zero_elements = (matrix !=0).sum().sum()
    sparsity_val=(total_elements-non_zero_elements)/total_elements
    return sparsity_val

sparsities_caltech = {user: sparsity(matrix, max_duration_caltech,max_energy_caltech) for user, matrix in joint_dists_caltech.items()}
sparsities_JPL = {user: sparsity(matrix, max_duration_JPL,max_energy_JPL) for user, matrix in joint_dists_JPL.items()}

In [ ]:
sparsities_JPL

#Calculate the ratio of entropy to sparsity for each user



In [ ]:
entropy_sparsity_caltech = {user: entropies_caltech[user] / sparsities_caltech[user] for user in joint_dists_caltech.keys()}
entropy_sparsity_JPL = {user: entropies_JPL[user] / sparsities_JPL[user] for user in joint_dists_JPL.keys()}

In [ ]:
entropy_sparsity_caltech

In [ ]:
entropy_sparsity_JPL